In [ ]:
import os
import sys
os.environ["WANDB_MODE"] = "disabled"

from omnisafe import Evaluator
from gym_navigation.envs.navigation_goal_safe import NavigationGoalSafe
from omnisafe.envs.wrapper import TimeLimit
import torch



save_dir = "runs/PPOLag-{NavigationGoalSafe-v0}/seed-000-2025-05-05-18-41-54"

available_files = [f for f in os.listdir(save_dir)]
print(f"Available files: {available_files}")

evaluator = Evaluator() 

evaluator.load_saved(
    save_dir=save_dir,
    model_name='epoch-100.pt',
)

evaluator._env = TimeLimit(evaluator._env, device=torch.device('cpu'), time_limit=1000)

evaluator.evaluate(num_episodes=1000)

In [ ]:
import os
import sys
import torch

# Override torch.load to always use CPU device before importing omnisafe
original_torch_load = torch.load
def cpu_torch_load(*args, **kwargs):
    if 'map_location' not in kwargs:
        kwargs['map_location'] = torch.device('cpu')
    return original_torch_load(*args, **kwargs)
torch.load = cpu_torch_load

os.environ["WANDB_MODE"] = "disabled"

# Now import omnisafe after patching torch.load
import omnisafe
from gym_navigation.envs.navigation_goal_safe import NavigationGoalSafe
from omnisafe.envs.wrapper import TimeLimit


save_dir = "runs/PPOLag-{NavigationGoalSafe-v0}/seed-000-2025-05-06-11-17-23"

# Force CPU device
device = torch.device('cpu')
torch.cuda.is_available = lambda: False

evaluator = omnisafe.Evaluator(render_mode='rgb_array')

save_replay_path = 'videos/' + save_dir.split('/')[1] + '/' + save_dir.split('/')[2] + '/'

# Load the saved model and render episodes
for item in os.scandir(os.path.join(save_dir, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        if item.name == 'epoch-50.pt': # only load the last checkpoint
            evaluator.load_saved(
                save_dir= save_dir,
                model_name=item.name,
                width=256,
                height=256,
                render_mode='rgb_array'
            )
            # limit the environment to 1000 steps
            evaluator._env = TimeLimit(evaluator._env, device=device, time_limit=1000)
            # save videos
            evaluator.render(num_episodes=10, save_replay_path=save_replay_path + "/" + item.name.split('.')[0])